# Salary Analytics

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly as pt

In [2]:
# constants
# source = https://docs.google.com/spreadsheets/d/19h0WT9Xjf9ncNmJDWpd8fzz7YnB37nemDnxKSO0EE8o/edit#gid=1664289965
file_path = 'Anonymous Salary Survey (Responses) - Form Responses 1.csv'

## Import Data

In [3]:
d = pd.read_csv(file_path)
d.shape

(59462, 22)

In [4]:
rename_cols = {
    'Age Range' : 'age_range',
    'Highest Level of Education Received' : 'max_edu',
    'Company Name' : 'org',
    'Years of Experience' : 'yoe',
    'Closest Major City and State (e.g. Santa Clara, CA)' : 'loc_city',
    'Annual Base Salary (if hourly, please convert to annual)' : 'annual_wage',
    'Currency (USD, CAD, etc)' : 'currency',
    'Gender (optional)' : 'gender',
    'Annual Bonus' : 'bonus',
    'Annual Average of RSUs' : 'avg_rsu',
    'Job Title' : 'job_title'
}
d.rename(columns = rename_cols, inplace = True)

In [5]:
d['is_ds'] = d['job_title'].apply(lambda x : [True if str(x).lower() in ['data', 'science', 'scientist', 'research'] else False][0])

In [13]:
d['annual_wage']

0        $85,000
1        $71,000
2        $97,000
3        $57,000
4        $80,000
          ...   
59457    110,000
59458     75,234
59459     74,000
59460      76950
59461    150,000
Name: annual_wage, Length: 59462, dtype: object

In [70]:
def clean_wage(wage):
    wage=str(wage).lower()
    wage = wage.replace('$','').replace(' ', '').replace(',', '').replace('cad', '').replace('usd','').replace('()','').replace('k', '000')
    try:
        return int(wage)
    except:
        return np.nan

In [71]:
d['parsed_annual_wage'] = d['annual_wage'].apply(lambda x : clean_wage(x))

In [32]:
d[d['is_ds']==True].parsed_annual_wage.max()

140000

In [35]:
d['gender'].value_counts()

Female                                                   44961
Male                                                     12478
Prefer Not to Say                                          290
Non-binary                                                  31
Nonbinary                                                   16
Non binary                                                  10
non-binary                                                  10
Non-binary                                                   9
Non binary                                                   7
Non-Binary                                                   5
non-binary                                                   5
NB                                                           3
nonbinary                                                    2
Nonbinary                                                    2
Non Binary                                                   2
Genderqueer                                            

In [34]:
d[d['parsed_annual_wage']==6969694206969696969596969]

,Timestamp,age_range,yoe,Industry,job_title,org,max_edu,loc_city,Country,annual_wage,...,How many vacation days are you given per year?,How many sick days are you given per year?,How many days per week are you required to work onsite/in the office?,Do you openly discuss salary with your colleagues?,How many months Maternity or Paternity does your company offer?,gender,Unnamed: 20,Diverse Identity (Optional),is_ds,parsed_annual_wage
34192,12/8/2021 7:41:26,58+,2,Straight up porn,The ultimate milker 💦,Jones bbq & foot massage,No Schooling Completed,"Dingleberry, WA",NaN,6969694206969696969596969,...,365,My immune system not like that fr,"None, I work fully remote",I be braggin n shi,Told them to keep it so I can focus on my pull...,Helicopter,NaN,Gendervoid foxkin demiqueer,False,6969694206969696969596969


In [56]:
def clean_currency(cur):
    cur = str(cur).lstrip().rstrip().strip().lower()
    if cur == 'nan':
        return np.nan
    return cur

In [58]:
d['currency'] = d['currency'].apply(lambda x : clean_currency(x))

In [59]:
d2 = d[d['parsed_annual_wage'].between(0, 1000000)]

In [65]:
d2[(d2.currency == 'cad') | (d2.currency == 'usd')].parsed_annual_wage.sort_values()

31252          0
30154          0
46558          0
12199          0
53656          0
          ...   
46056    1000000
45544    1000000
35646    1000000
1056     1000000
49472    1000000
Name: parsed_annual_wage, Length: 50130, dtype: object

In [66]:
d2[d2.parsed_annual_wage == 1000000]

,Timestamp,age_range,yoe,Industry,job_title,org,max_edu,loc_city,Country,annual_wage,...,How many vacation days are you given per year?,How many sick days are you given per year?,How many days per week are you required to work onsite/in the office?,Do you openly discuss salary with your colleagues?,How many months Maternity or Paternity does your company offer?,gender,Unnamed: 20,Diverse Identity (Optional),is_ds,parsed_annual_wage
1056,12/6/2021 23:10:22,38-41,15,Real Estate,Realtor,NaN,Bachelor's Degree,San Francisco,US,"$1,000,000",...,Unlimited,Unlimited,"None, I work fully remote",No,0,Female,NaN,NaN,False,1000000
31069,12/8/2021 1:45:39,30-33,12,Staffing,"Director, Client Services",NaN,Master's Degree,Seattle,US,"$1,000,000",...,21 days,NaN,"None, I work fully remote",No,1,Female,NaN,NaN,False,1000000
35646,12/8/2021 9:36:03,26-29,9,Management Consulting,Partner,Big 3 firm,Bachelor's Degree,"San Francisco, CA",US,"$1,000,000",...,20-30,NaN,4,No,NaN,Male,NaN,NaN,False,1000000
37293,12/8/2021 13:53:01,22-25,3,Business owner,CEO,Disclosed,Bachelor's Degree,NYC,US,"$1,000,000",...,Unlimited,When I need,7,No,When I need,Male,NaN,NaN,False,1000000
45544,12/9/2021 19:14:35,34-37,11,Pharmaceuticals,CEO,NaN,Bachelor's Degree,Zurich,Switzerland,"$1,000,000",...,As many as I want,As many as I need,3,No,NaN,Male,NaN,NaN,False,1000000
46056,12/9/2021 20:35:31,18-21,1,Tech,Software,Facebook,Bachelor's Degree,"New York, NY",Manhattan,"$1,000,000",...,365,365,1,Yes,12,Male,NaN,NaN,False,1000000
49472,12/11/2021 13:43:22,26-29,7,Market Research,Director of Business Development,NaN,Master's Degree,Washington DC,United States,"$1,000,000",...,15,5,"None, I work fully remote",No,3,Male,NaN,Hispanic/Latino,False,1000000
50783,12/14/2021 1:33:06,26-29,1,Social Media,TikTok Creator,NaN,Bachelor's Degree,"Ann Arbor, MI",USA,"$1,000,000",...,NaN,NaN,"None, I work fully remote",Yes,NaN,Male,NaN,White,False,1000000
57391,12/26/2021 5:22:04,22-25,1,edtech,UX Researcher,Byjus,"Some College credit, no degree",Kochi,India,"$1,000,000",...,24,12,"None, I work fully remote",No,3,Male,NaN,NaN,False,1000000


In [69]:
list(map(int, ['1', '2', '3']))

[1, 2, 3]

In [38]:
d.parsed_annual_wage.value_counts().reset_index().sort_values(by = 'index', ascending = False).head(50)

,index,parsed_annual_wage
1433,6969694206969696969596969,1
1861,99999999999999999999,1
2057,10000000000,1
2455,6520719382,1
1678,2000000000,1
3400,1750002021,1
2935,180000000,1
2915,74500000,1
1868,41000000,1
1982,40000000,1
